# One-hot encoder 

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/Erika-Ortiz/sesgos_diplodatos/main/datos_preprocesados.csv')

In [3]:
df.head()

,Participante,Modalidad,Edad,Género,Grupo,Validez_sil1,Validez_sil2,Validez_sil3,Validez_sil4,Validez_sil5,...,Aceptación_sil8,Correctas_sil1,Correctas_sil2,Correctas_sil3,Correctas_sil4,Correctas_sil5,Correctas_sil6,Correctas_sil7,Correctas_sil8,indice_creencia_norm
0,1,Resolución Individual,18,M,0,V,V,I,I,V,...,0,1,1,1,1,1,1,0,1,0.25
1,2,Resolución Individual,23,F,0,V,V,I,I,V,...,1,1,1,1,1,1,1,0,0,0.00
2,3,Resolución Individual,20,F,0,V,V,I,I,V,...,0,1,0,1,1,0,0,0,1,0.50
3,4,Resolución Individual,25,F,0,V,V,I,I,V,...,0,1,1,1,1,1,1,0,1,0.25
4,5,Resolución Individual,20,F,0,V,V,I,I,V,...,0,1,0,1,1,1,1,0,1,0.50


Separamos nuestra variable objetivo de las características que utilizaremos para entrenar nuestro modelo. 

Supongamos que deseamos hacer un entrenamiento usando las columnas de `Edad` y `Modalidad`: 

In [4]:
X = df[['Edad','Modalidad']]
y= df['indice_creencia_norm']

## División en conjuntos de train y test
Como les comentaba en el grupo, la división en train y test la hacemos antes de aplicar cualquier transformación en los datos. 

Les paso un [post](https://www.linkedin.com/feed/update/urn:li:activity:7102606015589699584?updateEntityUrn=urn%3Ali%3Afs_feedUpdate%3A%28V2%2Curn%3Ali%3Aactivity%3A7102606015589699584%29) de LinkedIn sobre el tema (Si tienen cuenta en esta red les recomiendo seguir a la autora del post porque da tips interesantes y muy instructivos)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Podemos ver la forma de nuestros conjuntos para luego chequear que, con las transformaciones, tengamos el número correcto de filas y columnas:

In [6]:
print(f"""Conjunto de train: {X_train.shape[0]} filas y {X_train.shape[1]} columnas.
Conjunto de test: {X_test.shape[0]} filas y {X_test.shape[1]} columnas. """)

Conjunto de train: 525 filas y 2 columnas.
Conjunto de test: 59 filas y 2 columnas. 


## Codificación de variables categóricas

La columna `Modalidad` contiene datos categóricos, veamos cómo codificarlos usando `OneHotEncoder` de scikit-learn.

Este método encuentra los valores únicos y transforma los datos a una codificación binaria. Sabemos, por lo que hemos trabajado en los prácticos anteriores, que tenemos cinco modalidades de resolución (valores únicos):

In [7]:
print(*X['Modalidad'].unique(), sep='\n')

Resolución Individual
Resolución Grupal
Grupal entre sujetos
Resolución Individual Post
Resolución Individual Pre


In [8]:
ohe_enc = OneHotEncoder(sparse_output=False)

In [9]:
mod_enc_train = ohe_enc.fit_transform(X_train[['Modalidad']])
mod_enc_test = ohe_enc.transform(X_test[['Modalidad']])

Podemos chequear que los arrays que se generan tienen la forma correcta:

In [10]:
print(mod_enc_train.shape)
print(mod_enc_test.shape)

(525, 5)
(59, 5)


En el paso siguiente, agregamos las columnas generadas por el encoder y eliminamos la columna original. 

Debemos prestar atención a los índices para evitar la generación de columnas o filas extras. Con este fin, en el siguiente código aplicamos `.reset_index(drop=True)`

In [11]:
X_train_encoded = pd.concat([X_train.drop('Modalidad', axis=1).reset_index(drop=True), 
                            pd.DataFrame(mod_enc_train)], axis=1)

X_test_encoded = pd.concat([X_test.drop('Modalidad', axis=1).reset_index(drop=True), 
                           pd.DataFrame(mod_enc_test)], axis=1)

Chequeamos si tenemos el número correcto de filas y columnas:

In [12]:
print(f"""Conjunto de train con columna categórica codificada: {X_train_encoded.shape[0]} filas y {X_train_encoded.shape[1]} columnas.
Conjunto de test con columna categórica codificada: {X_test_encoded.shape[0]} filas y {X_test_encoded.shape[1]} columnas. """)

Conjunto de train con columna categórica codificada: 525 filas y 6 columnas.
Conjunto de test con columna categórica codificada: 59 filas y 6 columnas. 


Vemos que el número de filas se mantiene y contamos con 6 columnas: una correspondiente a la columna `Edad` y, de las restantes cinco, una por cada valor único de la columna `Modalidad`: 

In [13]:
X_train_encoded.head()

,Edad,0,1,2,3,4
0,18,1.0,0.0,0.0,0.0,0.0
1,23,0.0,0.0,1.0,0.0,0.0
2,21,0.0,1.0,0.0,0.0,0.0
3,19,0.0,0.0,0.0,0.0,1.0
4,20,0.0,0.0,1.0,0.0,0.0


Ahora bien, notemos que los nombres de las columnas creadas por el codificador one-hot son números del 1 al 5. Si queremos que nuestras tengan como nombres los valores que encontrabamos en la columna original podemos utilizar el método `.categories_`: 

In [14]:
X_train_enc_2 = pd.concat([X_train['Edad'].reset_index(drop=True),
                              pd.DataFrame(mod_enc_train, columns = ohe_enc.categories_[0])],  axis = 1)
X_test_enc_2 = pd.concat([X_test['Edad'].reset_index(drop=True),
                              pd.DataFrame(mod_enc_test, columns = ohe_enc.categories_[0])],  axis = 1)

In [15]:
X_train_enc_2

,Edad,Grupal entre sujetos,Resolución Grupal,Resolución Individual,Resolución Individual Post,Resolución Individual Pre
0,18,1.0,0.0,0.0,0.0,0.0
1,23,0.0,0.0,1.0,0.0,0.0
2,21,0.0,1.0,0.0,0.0,0.0
3,19,0.0,0.0,0.0,0.0,1.0
4,20,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...
520,21,0.0,0.0,1.0,0.0,0.0
521,22,0.0,0.0,1.0,0.0,0.0
522,19,0.0,0.0,0.0,1.0,0.0
523,18,0.0,0.0,0.0,1.0,0.0
